In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

from sklearn.linear_model import LogisticRegression

In [2]:
def download(first_page, last_page):
    dfs = [] 
    for i in range(first_page, last_page):
        url = f'http://stats.espncricinfo.com/ci/engine/stats/index.html?class=11;orderby=start;page={i};template=results' \
              f';type=batting;view=innings;wrappertype=print'
        page = requests.get(url)
        html_table = BeautifulSoup(page.text, 'html.parser').find_all('table', class_='engineTable')[2]
        df = pd.read_html(str(html_table), header=0)[0]
        df = df.assign(Page = i)
        dfs.append(df)
    output = pd.concat(dfs, axis = 0)
    return(output)

In [9]:
dl = download(1, 3)
dl

,Player,Runs,Mins,BF,4s,6s,SR,Inns,Unnamed: 8,Opposition,Ground,Start Date,Page
0,C Bannerman (AUS),165*,285,-,18,0,-,1,NaN,Test v England,Melbourne,15 Mar 1877,1
1,NFD Thomson (AUS),1,-,-,0,0,-,1,NaN,Test v England,Melbourne,15 Mar 1877,1
2,TP Horan (AUS),12,-,-,-,0,-,1,NaN,Test v England,Melbourne,15 Mar 1877,1
3,DW Gregory (AUS),1,-,-,0,0,-,1,NaN,Test v England,Melbourne,15 Mar 1877,1
4,BB Cooper (AUS),15,-,-,-,0,-,1,NaN,Test v England,Melbourne,15 Mar 1877,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
45,T Emmett (ENG),0,1,1,0,0,0.00,1,NaN,Test v Australia,Melbourne,2 Jan 1879,2
46,CA Absolom (ENG),52,-,-,5,0,-,1,NaN,Test v Australia,Melbourne,2 Jan 1879,2
47,L Hone (ENG),7,-,-,-,0,-,1,NaN,Test v Australia,Melbourne,2 Jan 1879,2
48,SS Schultz (ENG),0*,-,-,0,0,-,1,NaN,Test v Australia,Melbourne,2 Jan 1879,2


In [21]:
df = dl \
    .assign(
        mt_start_date = lambda x: x['Start Date'].astype('datetime64[ns]'),
        mt_ground = lambda x: x['Ground'],
        mt_format = lambda x: x['Opposition'].str.extract(r'^(.*?)\sv'),
        mt_team_code = lambda x: x['Player'].str.extract(r'(?<=\()(.+?)(?=\))'),
        mt_opposition = lambda x: x['Opposition'].str.extract(r'v\s(.*)'),
        in_no = lambda x: x['Inns'].astype('int'),
        pl_name = lambda x: x['Player'].str.extract(r'^(.*?)\s\('),
        pf_runs = lambda x: np.where(x['Runs'] == 'DNB', 0, x['Runs'].str.replace(r'\*', '')).astype('int'),
        pf_balls = lambda x: np.where(x['Runs'] == 'DNB', 0, np.where(x['BF'] == '-', np.nan, x['BF'])).astype('float'),
        pf_mins = lambda x: np.where(x['Runs'] == 'DNB', 0, np.where(x['Mins'] == '-', np.nan, x['Mins'])).astype('float'),
        pf_fours = lambda x: np.where(x['Runs'] == 'DNB', 0, np.where(x['4s'] == '-', np.nan, x['4s'])).astype('float'),
        pf_sixes = lambda x: np.where(x['Runs'] == 'DNB', 0, np.where(x['6s'] == '-', np.nan, x['6s'])).astype('float'),
        pf_no = lambda x: np.where(x['Runs'].str.contains('*', regex=False), True, False).astype('bool'),
        pf_dnb = lambda x: np.where(x['Runs'] == 'DNB', True, False).astype('bool'),
        pf_highest_batsman = lambda x: x.groupby(['mt_start_date', 'mt_team_code', 'in_no'])['pf_runs'].transform('max') == x['pf_runs'],
        pl_batting_order = lambda x: x.groupby(['mt_start_date', 'mt_team_code', 'in_no']).cumcount() + 1,
        pl_inning_no = lambda x: x.sort_values(['mt_start_date', 'in_no']).groupby(['pl_name']).cumcount() + 1
    ) \
    .filter(like = '_') \
    .reset_index(drop = True) \
    .rename_axis('in_id')

df

,mt_start_date,mt_ground,mt_format,mt_team_code,mt_opposition,in_no,pl_name,pf_runs,pf_balls,pf_mins,pf_fours,pf_sixes,pf_no,pf_dnb,pf_highest_batsman,pl_batting_order,pl_inning_no
in_id,,,,,,,,,,,,,,,,,
0,1877-03-15,Melbourne,Test,AUS,England,1,C Bannerman,165,NaN,285.0,18.0,0.0,True,False,True,1,1
1,1877-03-15,Melbourne,Test,AUS,England,1,NFD Thomson,1,NaN,NaN,0.0,0.0,False,False,False,2,1
2,1877-03-15,Melbourne,Test,AUS,England,1,TP Horan,12,NaN,NaN,NaN,0.0,False,False,False,3,1
3,1877-03-15,Melbourne,Test,AUS,England,1,DW Gregory,1,NaN,NaN,0.0,0.0,False,False,False,4,1
4,1877-03-15,Melbourne,Test,AUS,England,1,BB Cooper,15,NaN,NaN,NaN,0.0,False,False,False,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1879-01-02,Melbourne,Test,ENG,Australia,1,T Emmett,0,1.0,1.0,0.0,0.0,False,False,False,8,5
96,1879-01-02,Melbourne,Test,ENG,Australia,1,CA Absolom,52,NaN,NaN,5.0,0.0,False,False,True,9,1
97,1879-01-02,Melbourne,Test,ENG,Australia,1,L Hone,7,NaN,NaN,NaN,0.0,False,False,False,10,1


# Logistic Regression

In [5]:
model = LogisticRegression(solver = 'lbfgs')

In [6]:
model.fit(df['pl_batting_order'].values.reshape(-1, 1), df['pf_highest_batsman'].values.astype('int'))

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [7]:
df = df
df['pr_highest_batsman'] = model.predict_proba(df['pl_batting_order'].values.reshape(-1, 1))[:, 1]
df

,mt_start_date,mt_ground,mt_format,mt_team_code,mt_opposition,in_no,pl_name,pf_runs,pf_balls,pf_mins,pf_fours,pf_sixes,pf_no,pf_dnb,pf_highest_batsman,pl_batting_order,pl_inning_no,pr_highest_batsman
0,1877-03-15,Melbourne,Test,AUS,England,1,C Bannerman,165,NaN,285.0,18.0,0.0,True,False,True,1,1,0.229282
1,1877-03-15,Melbourne,Test,AUS,England,1,NFD Thomson,1,NaN,NaN,0.0,0.0,False,False,False,2,1,0.188657
2,1877-03-15,Melbourne,Test,AUS,England,1,TP Horan,12,NaN,NaN,NaN,0.0,False,False,False,3,1,0.153793
3,1877-03-15,Melbourne,Test,AUS,England,1,DW Gregory,1,NaN,NaN,0.0,0.0,False,False,False,4,1,0.124385
4,1877-03-15,Melbourne,Test,AUS,England,1,BB Cooper,15,NaN,NaN,NaN,0.0,False,False,False,5,1,0.099936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1879-01-02,Melbourne,Test,ENG,Australia,1,T Emmett,0,1.0,1.0,0.0,0.0,False,False,False,8,5,0.050349
96,1879-01-02,Melbourne,Test,ENG,Australia,1,CA Absolom,52,NaN,NaN,5.0,0.0,False,False,True,9,1,0.039791
97,1879-01-02,Melbourne,Test,ENG,Australia,1,L Hone,7,NaN,NaN,NaN,0.0,False,False,False,10,1,0.031374
98,1879-01-02,Melbourne,Test,ENG,Australia,1,SS Schultz,0,NaN,NaN,0.0,0.0,True,False,False,11,1,0.024691


In [22]:
model.score(df['pl_batting_order'].values.reshape(-1, 1), df['pf_highest_batsman'].values)

NameError: name 'model' is not defined

# Deep Learning

In [9]:
df.sort_values(['pl_name', 'mt_start_date'])

,mt_start_date,mt_ground,mt_format,mt_team_code,mt_opposition,in_no,pl_name,pf_runs,pf_balls,pf_mins,pf_fours,pf_sixes,pf_no,pf_dnb,pf_highest_batsman,pl_batting_order,pl_inning_no,pr_highest_batsman
15,1877-03-15,Melbourne,Test,ENG,Australia,2,A Greenwood,1,26.0,22.0,0.0,0.0,False,False,False,5,1,0.099936
34,1877-03-15,Melbourne,Test,ENG,Australia,4,A Greenwood,5,5.0,5.0,0.0,0.0,False,False,False,2,2,0.188657
57,1877-03-31,Melbourne,Test,ENG,Australia,2,A Greenwood,49,NaN,NaN,NaN,0.0,False,False,False,3,3,0.153793
79,1877-03-31,Melbourne,Test,ENG,Australia,4,A Greenwood,22,NaN,NaN,NaN,0.0,False,False,False,3,4,0.153793
19,1877-03-15,Melbourne,Test,ENG,Australia,2,A Hill,35,47.0,41.0,3.0,0.0,True,False,False,9,1,0.039791
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,1877-03-15,Melbourne,Test,AUS,England,3,WE Midwinter,17,73.0,68.0,1.0,0.0,False,False,False,6,2,0.079854
49,1877-03-31,Melbourne,Test,AUS,England,1,WE Midwinter,31,NaN,NaN,NaN,0.0,False,False,True,6,3,0.079854
71,1877-03-31,Melbourne,Test,AUS,England,3,WE Midwinter,12,NaN,NaN,NaN,0.0,False,False,False,6,4,0.079854
51,1877-03-31,Melbourne,Test,AUS,England,1,WL Murdoch,3,NaN,NaN,0.0,0.0,False,False,False,8,1,0.050349


In [14]:
test = df.copy() \
  .filter(['pl_name', 'pl_inning_no', 'mt_start_date', 'pf_runs']) \
  .sort_values(['pl_name', 'pl_inning_no'])

test

,pl_name,pl_inning_no,mt_start_date,pf_runs
15,A Greenwood,1,1877-03-15,1
34,A Greenwood,2,1877-03-15,5
57,A Greenwood,3,1877-03-31,49
79,A Greenwood,4,1877-03-31,22
19,A Hill,1,1877-03-15,35
...,...,...,...,...
27,WE Midwinter,2,1877-03-15,17
49,WE Midwinter,3,1877-03-31,31
71,WE Midwinter,4,1877-03-31,12
51,WL Murdoch,1,1877-03-31,3


In [59]:
test.pivot(index = 'pl_name', columns = 'pl_inning_no').swaplevel(axis = 1).sort_index(axis = 1, level = 'pl_inning_no')

pl_inning_no                     1                     2          \
                     mt_start_date pf_runs mt_start_date pf_runs   
pl_name                                                            
A Greenwood             1877-03-15     1.0    1877-03-15     5.0   
A Hill                  1877-03-15    35.0    1877-03-15     0.0   
A Shaw                  1877-03-15    10.0    1877-03-15     2.0   
AJ Webbe                1879-01-02     4.0           NaT     NaN   
AN Hornby               1879-01-02     2.0           NaT     NaN   
AP Lucas                1879-01-02     6.0           NaT     NaN   
BB Cooper               1877-03-15    15.0    1877-03-15     3.0   
C Bannerman             1877-03-15   165.0    1877-03-15     4.0   
CA Absolom              1879-01-02    52.0           NaT     NaN   
DW Gregory              1877-03-15     1.0    1877-03-15     3.0   
EJ Gregory              1877-03-15     0.0    1877-03-15    11.0   
FA MacKinnon            1879-01-02     0.0           NaT     NaN   
FR Spofforth            1877-03-31     0.0    1877-03-31    17.0   
G Ulyett                1877-03-15    10.0    1877-03-15    24.0   
H Jupp                  1877-03-15    63.0    1877-03-15     4.0   
HRJ Charlwood           1877-03-15    36.0    1877-03-15    13.0   
J Selby                 1877-03-15     7.0    1877-03-15    38.0   
J Southerton            1877-03-15     6.0    1877-03-15     1.0   
JM Blackham             1877-03-15    17.0    1877-03-15     6.0   
JR Hodges               1877-03-15     0.0    1877-03-15     8.0   
James Lillywhite jnr    1877-03-15    10.0    1877-03-15     4.0   
L Hone                  1879-01-02     7.0           NaT     NaN   
Lord Harris             1879-01-02    33.0           NaT     NaN   
NFD Thomson             1877-03-15     1.0    1877-03-15     7.0   
SS Schultz              1879-01-02     0.0           NaT     NaN   
T Armitage              1877-03-15     9.0    1877-03-15     3.0   
T Emmett                1877-03-15     8.0    1877-03-15     9.0   
TJD Kelly               1877-03-31    19.0    1877-03-31    35.0   
TK Kendall              1877-03-15     3.0    1877-03-15    17.0   
TP Horan                1877-03-15    12.0    1877-03-15    20.0   
TW Garrett              1877-03-15    18.0    1877-03-15     0.0   
VPFA Royle              1879-01-02     3.0           NaT     NaN   
WE Midwinter            1877-03-15     5.0    1877-03-15    17.0   
WL Murdoch              1877-03-31     3.0    1877-03-31     8.0   

pl_inning_no                     3                     4          \
                     mt_start_date pf_runs mt_start_date pf_runs   
pl_name                                                            
A Greenwood             1877-03-31    49.0    1877-03-31    22.0   
A Hill                  1877-03-31    49.0    1877-03-31    17.0   
A Shaw                  1877-03-31     1.0    1877-03-31     0.0   
AJ Webbe                       NaT     NaN           NaT     NaN   
AN Hornby                      NaT     NaN           NaT     NaN   
AP Lucas                       NaT     NaN           NaT     NaN   
BB Cooper                      NaT     NaN           NaT     NaN   
C Bannerman             1877-03-31    10.0    1877-03-31    30.0   
CA Absolom                     NaT     NaN           NaT     NaN   
DW Gregory              1877-03-31     1.0    1877-03-31    43.0   
EJ Gregory                     NaT     NaN           NaT     NaN   
FA MacKinnon                   NaT     NaN           NaT     NaN   
FR Spofforth                   NaT     NaN           NaT     NaN   
G Ulyett                1877-03-31    52.0    1877-03-31    63.0   
H Jupp                  1877-03-31     0.0    1877-03-31     1.0   
HRJ Charlwood           1877-03-31    14.0    1877-03-31     0.0   
J Selby                 1877-03-31     7.0    1877-03-31     2.0   
J Southerton            1877-03-31     0.0    1877-03-31     0.0   
JM Blackham             1877-03-31     5.0    1877-03-3